In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import numpy as np
pd.options.display.max_rows = 200

%matplotlib inline

In [2]:
files = glob.glob('../google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/*')
[print(i,'-->',f) for i,f in enumerate(files)];

0 --> ../google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/MNCAATourneyDetailedResults.csv
1 --> ../google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/MNCAATourneyCompactResults.csv
2 --> ../google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/MSeasons.csv
3 --> ../google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/MRegularSeasonDetailedResults.csv
4 --> ../google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/MNCAATourneySlots.csv
5 --> ../google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/MGameCities.csv
6 --> ../google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/MConferenceTourneyGames.csv
7 --> ../google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/Cities.csv
8 --> ../google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_S

# Season Stats

In [3]:
def loc_func(x):
    if x['WLoc'] == 'N':
        return 'N'
    elif x['WLoc'] == 'H' and x['result'] == 1:
        return 'H'
    elif x['WLoc'] == 'A' and x['result'] == 1:
        return 'A'
    elif x['WLoc'] == 'H' and x['result'] == 0:
        return 'A'
    elif x['WLoc'] == 'A' and x['result'] == 0:
        return 'H'
    else:
        return None

In [4]:
SeasonResults = pd.read_csv(files[8])
SeasonResults['GameID'] = np.arange(1,len(SeasonResults)+1)
SeasonResultsWin = SeasonResults[['Season','DayNum','GameID','WTeamID','WScore','LTeamID','LScore','WLoc']].copy()
SeasonResultsWin.columns = ['Season','DayNum','GameID','TeamID','Score','TeamID_OPP','Score_OPP','WLoc']
SeasonResultsWin['result'] = 1
SeasonResultsLose = SeasonResults[['Season','DayNum','GameID','WTeamID','WScore','LTeamID','LScore','WLoc']].copy()
SeasonResultsLose.columns = ['Season','DayNum','GameID','TeamID_OPP','Score_OPP','TeamID','Score','WLoc']
SeasonResultsLose['result'] = 0
SeasonResultsTeams = pd.concat([SeasonResultsWin,SeasonResultsLose],axis=0,sort=True).sort_values('GameID')
SeasonResultsTeams['Loc'] = SeasonResultsTeams.apply(loc_func,axis=1)

seasonAve = (SeasonResultsTeams
             .groupby(['Season','TeamID'])[['Score','Score_OPP','result']]
             .mean()
             .rename({'result':'WinPCT'},axis=1)
             .reset_index()
            )
seasonAve['ScoreDiff'] = seasonAve['Score'] - seasonAve['Score_OPP']

seasonAve.head(10)

,Season,TeamID,Score,Score_OPP,WinPCT,ScoreDiff
0,1985,1102,63.083333,68.875000,0.208333,-5.791667
1,1985,1103,61.043478,64.086957,0.391304,-3.043478
2,1985,1104,68.500000,60.700000,0.700000,7.800000
3,1985,1106,71.625000,75.416667,0.416667,-3.791667
4,1985,1108,83.000000,75.040000,0.760000,7.960000
5,1985,1109,53.833333,82.958333,0.041667,-29.125000
6,1985,1110,69.440000,77.200000,0.280000,-7.760000
7,1985,1111,67.125000,69.916667,0.416667,-2.791667
8,1985,1112,66.518519,59.333333,0.666667,7.185185
9,1985,1113,67.703704,71.777778,0.407407,-4.074074


# Rankings

In [5]:
ranking = pd.read_csv(files[12])
#rank_methods = ['COL','DOL','MOR','POM','RTH','SAG','WLK','WOL']
rank_methods = ['POM']
team_rank = (ranking[(ranking['RankingDayNum']==133)&(ranking['SystemName'].isin(rank_methods))]
             .groupby(['Season','TeamID','SystemName'])['OrdinalRank']
             .mean()
             .unstack(2)
             .reset_index()
            )
team_rank.head(10)

SystemName,Season,TeamID,POM
0,2003,1102,160.0
1,2003,1103,163.0
2,2003,1104,33.0
3,2003,1105,307.0
4,2003,1106,263.0
5,2003,1107,312.0
6,2003,1108,279.0
7,2003,1110,143.0
8,2003,1111,170.0
9,2003,1112,3.0


# Tournament Seeds

In [6]:
TourneySeeds = pd.read_csv(files[15])
TourneySeeds['SeedN'] = TourneySeeds['Seed'].apply(lambda x: int(x[1:3]))
TourneySeeds.drop('Seed',axis=1,inplace=True)
TourneySeeds.head(10)

,Season,TeamID,SeedN
0,1985,1207,1
1,1985,1210,2
2,1985,1228,3
3,1985,1260,4
4,1985,1374,5
5,1985,1208,6
6,1985,1393,7
7,1985,1396,8
8,1985,1439,9
9,1985,1177,10


# Team Data
Merge all the teams information (per season) into one file

In [7]:
teamData = (seasonAve
            .merge(team_rank,on=['Season','TeamID'],how='left')
            .merge(TourneySeeds,on=['Season','TeamID'],how='left')
           )
teamData

,Season,TeamID,Score,Score_OPP,WinPCT,ScoreDiff,POM,SeedN
0,1985,1102,63.083333,68.875000,0.208333,-5.791667,NaN,NaN
1,1985,1103,61.043478,64.086957,0.391304,-3.043478,NaN,NaN
2,1985,1104,68.500000,60.700000,0.700000,7.800000,NaN,7.0
3,1985,1106,71.625000,75.416667,0.416667,-3.791667,NaN,NaN
4,1985,1108,83.000000,75.040000,0.760000,7.960000,NaN,NaN
...,...,...,...,...,...,...,...,...
11236,2019,1462,71.878788,70.606061,0.545455,1.272727,64.0,NaN
11237,2019,1463,80.892857,73.714286,0.750000,7.178571,82.0,14.0
11238,2019,1464,73.500000,79.233333,0.333333,-5.733333,263.0,NaN
11239,2019,1465,75.461538,75.192308,0.461538,0.269231,192.0,NaN


# Tournament Data
Merge the teams data into the tournement data file.
TeamID1 - always the team with the lower TeamID
TeamID2 - always the team with the higher TeamID
result - for TeamID1

In [8]:
TourneyCompactResults = pd.read_csv(files[1])
TourneyCompactResults['TeamID1'] = np.minimum(TourneyCompactResults['WTeamID'],TourneyCompactResults['LTeamID'])
TourneyCompactResults['TeamID2'] = np.maximum(TourneyCompactResults['WTeamID'],TourneyCompactResults['LTeamID'])
TourneyCompactResults['result'] = np.where(TourneyCompactResults['WTeamID']==TourneyCompactResults['TeamID1'],1,0)
TourneyCompactResults['ID'] = TourneyCompactResults['Season'].astype(str)+ '_' +TourneyCompactResults['TeamID1'].astype(str)+ '_' +TourneyCompactResults['TeamID2'].astype(str)
    
TourneyCompactResults = (TourneyCompactResults
                         .merge(teamData,left_on=['Season','TeamID1'],right_on=['Season','TeamID'],how='left')
                         .drop('TeamID',axis=1)
                         .merge(teamData,left_on=['Season','TeamID2'],right_on=['Season','TeamID'],how='left')
                         .drop('TeamID',axis=1)
                        )

TourneyCompactResults['SeedDiff'] = TourneyCompactResults['SeedN_x'] - TourneyCompactResults['SeedN_y']
TourneyCompactResults['ScoreDiff'] = TourneyCompactResults['ScoreDiff_x']  - TourneyCompactResults['ScoreDiff_y']
TourneyCompactResults['POMDiff'] = TourneyCompactResults['POM_x']  - TourneyCompactResults['POM_y']
TourneyCompactResults['WinPCTDiff'] = TourneyCompactResults['WinPCT_x']  - TourneyCompactResults['WinPCT_y']

TourneyCompactResults.head().T

,0,1,2,3,4
Season,1985,1985,1985,1985,1985
DayNum,136,136,136,136,136
WTeamID,1116,1120,1207,1229,1242
WScore,63,59,68,58,49
LTeamID,1234,1345,1250,1425,1325
LScore,54,58,43,55,38
WLoc,N,N,N,N,N
NumOT,0,0,0,0,0
TeamID1,1116,1120,1207,1229,1242
TeamID2,1234,1345,1250,1425,1325


# TRAIN

In [9]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupKFold
from sklearn.metrics import log_loss
from sklearn import preprocessing
from sklearn.inspection import permutation_importance
from sklearn.ensemble import StackingClassifier

In [10]:
cols = ['Score_x',
 'Score_OPP_x',
 'ScoreDiff_x',
 'POM_x',
 'SeedN_x',
 'Score_y',
 'Score_OPP_y',
 'ScoreDiff_y',
 'POM_y',
 'SeedN_y',
 'SeedDiff',
 'ScoreDiff',
 'POMDiff',
 'WinPCT_x',
 'WinPCT_y',
 'WinPCTDiff']

In [11]:
X = TourneyCompactResults.loc[:,cols]
y = TourneyCompactResults[['result']].values.ravel()

min_year = 2002

# Train Models
Method 1
use all the previous seasons to predict a particular season

In [12]:
rf = RandomForestClassifier(n_estimators=100,random_state=42,min_samples_split=100)
lr = LogisticRegression(solver='lbfgs',C=0.1,random_state=42,max_iter=1000)
lsvc = SVC(random_state=42,probability=True,kernel='linear')
nnb = KNeighborsClassifier(50)

estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('lr',LogisticRegression(solver='lbfgs',C=0.1,random_state=42,max_iter=500)),
    ('nnb',KNeighborsClassifier(50)),
    ('lsvc',SVC(random_state=42,probability=True,kernel='linear'))]
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

scores = np.zeros((5,5))
for ii,s in enumerate(range(2010,2015)):
    idxTrain = ((TourneyCompactResults['Season'] < s) & (TourneyCompactResults['Season'] > min_year))
    idxTest = (TourneyCompactResults['Season'] == s)
    # fit all models
    rf.fit(X.loc[idxTrain],y[idxTrain])
    lr.fit(X.loc[idxTrain],y[idxTrain])
    lsvc.fit(X.loc[idxTrain],y[idxTrain])
    nnb.fit(X.loc[idxTrain],y[idxTrain])
    clf.fit(X.loc[idxTrain],y[idxTrain])
    
    ypred_rf = rf.predict_proba(X.loc[idxTest])
    ypred_lr = lr.predict_proba(X.loc[idxTest])
    ypred_lsvc = lsvc.predict_proba(X.loc[idxTest])
    ypred_nnb = nnb.predict_proba(X.loc[idxTest])
    ypred_clf = clf.predict_proba(X.loc[idxTest])
    
    scores[ii,0] = log_loss(y[idxTest],ypred_rf[:,1])
    scores[ii,1] = log_loss(y[idxTest],ypred_lr[:,1])
    scores[ii,2] = log_loss(y[idxTest],ypred_lsvc[:,1])
    scores[ii,3] = log_loss(y[idxTest],ypred_nnb[:,1])
    scores[ii,4] = log_loss(y[idxTest],ypred_clf[:,1])

/Users/apple/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [13]:
scores_df = pd.DataFrame(scores,
                         index = range(2010,2015),
                         columns=['random forest','logistic regression','linear svc','nearest neighbor','stacking']).T

scores_df['average score'] = scores_df.mean(axis=1)
scores_df['score std'] = scores_df.std(axis=1)

scores_df

,2010,2011,2012,2013,2014,average score,score std
random forest,0.556771,0.669634,0.560905,0.626996,0.598192,0.602500,0.042301
logistic regression,0.554581,0.643976,0.581570,0.652677,0.597877,0.606136,0.037222
linear svc,0.558594,0.620481,0.561626,0.655132,0.630711,0.605309,0.038596
nearest neighbor,0.594839,0.617833,0.557865,0.641447,0.588438,0.600084,0.028184
stacking,0.572163,0.648243,0.555376,0.650369,0.616779,0.608586,0.038842


# Create groups by seasons

In [14]:
idxTrain = ((TourneyCompactResults['Season'] < 2015) & (TourneyCompactResults['Season'] > min_year))
group_kfold = GroupKFold(n_splits=6)

scores = []
for model in [rf,lr,lsvc,nnb,clf]:
    scores.append(cross_val_score(model,
                            X[idxTrain],
                            y[idxTrain],
                            groups=TourneyCompactResults.loc[idxTrain,'Season'],
                            cv=group_kfold,
                            scoring='neg_log_loss'))

In [15]:
pd.DataFrame([-1*np.mean(np.array(scores),axis=1),np.std(np.array(scores),axis=1)],
            columns=['random forest','logistic regression','linear svc','nearest neighbor','stacking'],
            index = ['average score','score std']).T

,average score,score std
random forest,0.558016,0.038805
logistic regression,0.551658,0.037883
linear svc,0.554340,0.040589
nearest neighbor,0.568346,0.031287
stacking,0.560910,0.033530


In [21]:
idxTrain = ((TourneyCompactResults['Season'] < 2015) & (TourneyCompactResults['Season'] > min_year))
lr.fit(X.loc[idxTrain],y[idxTrain]);
rf.fit(X.loc[idxTrain],y[idxTrain]);

In [22]:
testDF = pd.read_csv('../google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MSampleSubmissionStage1_2020.csv').drop('Pred',axis=1)
testDF = pd.concat([testDF,testDF['ID'].str.split('_',expand=True).rename({0:'Season',1:'TeamID1',2:'TeamID2'},axis=1)],axis=1)
testDF['Season'] = testDF['Season'].astype('int64')
testDF['TeamID1'] = testDF['TeamID1'].astype('int64')
testDF['TeamID2'] = testDF['TeamID2'].astype('int64')
testDF.head(10)

,ID,Season,TeamID1,TeamID2
0,2015_1107_1112,2015,1107,1112
1,2015_1107_1116,2015,1107,1116
2,2015_1107_1124,2015,1107,1124
3,2015_1107_1125,2015,1107,1125
4,2015_1107_1129,2015,1107,1129
5,2015_1107_1138,2015,1107,1138
6,2015_1107_1139,2015,1107,1139
7,2015_1107_1140,2015,1107,1140
8,2015_1107_1153,2015,1107,1153
9,2015_1107_1157,2015,1107,1157


In [24]:
subFile = (testDF
             .merge(teamData,left_on=['Season','TeamID1'],right_on=['Season','TeamID'],how='left')
             .drop('TeamID',axis=1)
             .merge(teamData,left_on=['Season','TeamID2'],right_on=['Season','TeamID'],how='left')
             .drop('TeamID',axis=1)
          )

subFile['SeedDiff'] = subFile['SeedN_x'] - subFile['SeedN_y']
subFile['ScoreDiff'] = subFile['ScoreDiff_x']  - subFile['ScoreDiff_y']
subFile['POMDiff'] = subFile['POM_x']  - subFile['POM_y']
subFile['WinPCTDiff'] = subFile['WinPCT_x']  - subFile['WinPCT_y']

subFile['Pred1'] = lr.predict_proba(subFile.loc[:,cols])[:,1]
subFile['Pred2'] = rf.predict_proba(subFile.loc[:,cols])[:,1]

subFile['Pred'] = 0.5*(subFile['Pred1']+subFile['Pred2'])
subFile = subFile[['ID','Pred']]

subFile.to_csv('results.csv',index=False)